In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer
from bs4 import BeautifulSoup
from io import StringIO
import csv
from tqdm import tqdm
import pandas as pd
import numpy as np

/mnt/gime-extract/miniconda3/envs/llmenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence"]

model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
embeddings = model.encode(sentences)

In [3]:
data = pd.read_csv('../data/pseudo_json_dataset_V3_2048.csv')

In [4]:
output_df = pd.DataFrame(columns=['index', 'question', 'csv_row', 'cosine_similarity', 'dot_product_similarity'])

In [5]:
index_df = pd.read_csv('../data/index_info.csv')
index_df.head()

,Index of data without headers,Index of data with multiple headers,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,1,692.0,NaN,NaN,
1,36,712.0,NaN,NaN,NaN
2,60,NaN,NaN,NaN,NaN
3,90,NaN,NaN,NaN,NaN
4,111,NaN,NaN,NaN,NaN


In [6]:
index_info = index_df['Index of data without headers'].to_list()

In [7]:
# with open(filename, 'w') as file:
count = 0
for i, row in tqdm(data[:2000].iterrows()):
    soup = BeautifulSoup(row['html_content'], 'html.parser')

    # Find the table
    table = soup.find('table')
    # Open StringIO object for writing
    csv_buffer = StringIO()


    trows = table.find_all('tr')
    question_embed = model.encode(row['question'])
    
    if row['index'] not in index_info: 
        header = trows[0]

        for trow in trows[1:]:
            writer = csv.writer(csv_buffer)
            writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])
            csv_data = csv_buffer.getvalue().replace('\n', '').replace('  ', '')
            csv_data_embed = model.encode(csv_data)
            csv_buffer = StringIO()
            sim = np.dot(csv_data_embed, question_embed)
            cosine_sim = cosine_similarity(csv_data_embed.reshape(1, -1), question_embed.reshape(1, -1))[0][0]
            output_df.at[count, 'question'] = row['question']
            output_df.at[count, 'cosine_similarity'] = cosine_sim
            output_df.at[count, 'dot_product_similarity'] = sim
            output_df.at[count, 'csv_row'] = csv_data
            output_df.at[count, 'index'] = row['index']
            count+=1
    else:
        for trow in trows:
            writer = csv.writer(csv_buffer)
            writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])
            csv_data = csv_buffer.getvalue().replace('\n', '').replace('  ', '')
            csv_data_embed = model.encode(csv_data)
            csv_buffer = StringIO()
            sim = np.dot(csv_data_embed, question_embed)
            cosine_sim = cosine_similarity(csv_data_embed.reshape(1, -1), question_embed.reshape(1, -1))[0][0]
            output_df.at[count, 'question'] = row['question']
            output_df.at[count, 'cosine_similarity'] = cosine_sim
            output_df.at[count, 'dot_product_similarity'] = sim
            output_df.at[count, 'csv_row'] = csv_data
            output_df.at[count, 'index'] = row['index']
            count+=1
        
        # writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])


    # Get the CSV data as a string
    

1it [00:00,  9.91it/s]

2000it [03:14, 10.28it/s]


In [8]:
output_df.to_csv('../embedding_similarity/sentence_transformers/bert_base_nli_mean_tokens_v1.csv', index=False)

In [9]:
model_id = 'RUCKBReasoning/TableLLM-7b'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [10]:
# with open(filename, 'w') as file:
count = 0
for i, row in tqdm(data[:2000].iterrows()):
    soup = BeautifulSoup(row['html_content'], 'html.parser')

    # Find the table
    table = soup.find('table')
    # Open StringIO object for writing
    csv_buffer = StringIO()


    trows = table.find_all('tr')
    header = trows[0]
    question_embed = tokenizer.encode(row['question'])

    for trow in trows[1:]:
        writer = csv.writer(csv_buffer)
        writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])
        csv_data = csv_buffer.getvalue()
        csv_data_embed = tokenizer.encode(csv_data)
        csv_buffer = StringIO()
        sim = np.dot(csv_data_embed, question_embed)
        cosine_sim = cosine_similarity(csv_data_embed.reshape(1, -1), question_embed.reshape(1, -1))[0][0]
        output_df.at[count, 'question'] = row['question']
        output_df.at[count, 'cosine_similarity'] = cosine_sim
        output_df.at[count, 'dot_product_similarity'] = sim
        output_df.at[count, 'csv_row'] = csv_data.replace('\n', '').replace('  ', '')
        output_df.at[count, 'index'] = row['index']
        count+=1
        # writer.writerow([cell.get_text() for cell in trow.find_all(['td', 'th'])])


    # Get the CSV data as a string
    

0it [00:00, ?it/s]


ValueError: shapes (52,) and (20,) not aligned: 52 (dim 0) != 20 (dim 0)

: 

In [ ]:
a = [2, 3, 2, 56, 78, 99]
b = [3, 44, 2, 2, 5, 99, 22, 33, 44, 66, 88, 99]

# Count occurrences of elements in list a
a_counts = {}
for item in a:
    a_counts[item] = a_counts.get(item, 0) + 1

# Count occurrences of elements in list b
b_counts = {}
for item in b:
    b_counts[item] = b_counts.get(item, 0) + 1

# Find common elements and count their occurrences
common_values_count = 0
for item, count in a_counts.items():
    if item in b_counts:
        common_values_count += min(count, b_counts[item])

print("Count of the same values in both lists:", common_values_count)

Count of the same values in both lists: 4
